<a href="https://colab.research.google.com/github/franciscogarate/cdiae/blob/main/notebooks/11_Bootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cálculo de la media y varianza valiéndose de la técnica de remuestreo (bootstrap) obteniendo 10000 $\overline{s}$.

En el anexo XVII del Reglamento Delegado, figuran los requisitos y demostraciones que deben realizarse para la aprobación de parámetros específicos de empresa. Entre ellos, se debe demostrar que la siniestralidad agregada sigue una distribución logarítmica normal $LN(\mu,\sigma)$.
Se pide, partiendo de los siniestros recogidos en el fichero *siniestralidad_agregada.csv*:


Importamos las librerías que vamos a necesitar

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt

Cargamos el fichero de siniestralidad agregada

In [ ]:
!git clone https://github.com/franciscogarate/cdiae

In [ ]:
df = pd.read_csv('cdiae/data/01_raw/siniestralidad_agregada.csv', sep=';', decimal=',')
df.head()

Calculamos la siniestralidad agregada (suma de importes)

In [ ]:
siniestralidad_agregada = df['Importe'].sum()
print(f'Siniestralidad agregada: {siniestralidad_agregada:,.0f}')

Aplicamos bootstrap: generamos 10000 remuestreos de la siniestralidad total

In [ ]:
n = df.Importe.count()
b = 10000

sampling = pd.DataFrame(data=np.zeros(b),columns=['s'])
for i in range(b):
    sample = df.sample(n, replace=True)
    summ = sample['Importe'].sum()
    sampling.loc[i, 's'] = summ

sampling['z'] = np.log(sampling['s'])
boot_mean = sampling['z'].mean()
boot_std = sampling['z'].std()

Mostramos los parámetros estimados de la distribución lognormal, que serían nuestras variables candidatas LN(mu=boot_mean, sigma=boot_stp)

In [ ]:
print(boot_mean, boot_std)

### Cálculo de  los principales percentiles (.25, .5, 0.75, 0.90, 0.95) de la distribución teórica y compararlos con los datos de la distribución empírica de los siniestros.

NOTA: siendo s la siniestralidad total y z la variable aleatoria normal, entonces $s = e^{z}$ y $z = log(s)$ con media y varianza:

$E(s) = e^{\mu+\frac{\sigma^2}{2}}$

$Var(s) = (e^{\sigma^2} -1) e^{2\mu+\sigma^2}$

Comparamos percentiles teóricos vs empíricos de s

In [ ]:
emp_q = sampling['s'].quantile([0.25, 0.5, 0.75, 0.90, 0.95])

# Percentiles teóricos de la lognormal ajustada
p = np.array([0.25, 0.50, 0.75, 0.90, 0.95])
theo_q = st.lognorm.ppf(p, s=sigma, scale=np.exp(mu))

pct_table = pd.DataFrame({
    'p': p,
    'empirico_s': emp_q.values,
    'teorico_s': theo_q
})

pct_table.round(5)

In [ ]:
xmin = sampling['s'].quantile(0.01)
xmax = sampling['s'].quantile(0.99)
xs = np.linspace(xmin, xmax, 400)

# Histograma de densidad de s
plt.hist(sampling['s'], bins=40, density=True, alpha=0.5)
# PDF teórica
pdf = st.lognorm.pdf(xs, s=sigma, scale=np.exp(mu))
plt.plot(xs, pdf)
plt.show()